These are the standard models used in the publication "Generalizability Assessment of AI Models Across Hospitals: A Comparative Study in Low-Middle Income and High Income Countries" (https://www.medrxiv.org/content/10.1101/2023.11.05.23298109v1).

The 

## Load dataset

In [28]:
#This script automatically loads the Adult (Census) dataset (replace with your own data)

import pandas as pd

# Define column names
column_names = ['age', 'workclass', 'fnlwgt', 'education', 'education-num', 'marital-status', 'occupation',
                'relationship', 'race', 'sex', 'capital-gain', 'capital-loss', 'hours-per-week', 'native-country', 'income']

# Load dataset
url = 'https://archive.ics.uci.edu/ml/machine-learning-databases/adult/adult.data'
df = pd.read_csv(url, header=None, names=column_names)

# Display the first few rows of the dataset
print(df.head())

   age          workclass  fnlwgt   education  education-num  \
0   39          State-gov   77516   Bachelors             13   
1   50   Self-emp-not-inc   83311   Bachelors             13   
2   38            Private  215646     HS-grad              9   
3   53            Private  234721        11th              7   
4   28            Private  338409   Bachelors             13   

        marital-status          occupation    relationship    race      sex  \
0        Never-married        Adm-clerical   Not-in-family   White     Male   
1   Married-civ-spouse     Exec-managerial         Husband   White     Male   
2             Divorced   Handlers-cleaners   Not-in-family   White     Male   
3   Married-civ-spouse   Handlers-cleaners         Husband   Black     Male   
4   Married-civ-spouse      Prof-specialty            Wife   Black   Female   

   capital-gain  capital-loss  hours-per-week  native-country  income  
0          2174             0              40   United-States   <=50

In [31]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler

# Encode categorical variables
label_encoders = {}
for column in df.select_dtypes(include=['object']).columns:
    label_encoders[column] = LabelEncoder()
    df[column] = label_encoders[column].fit_transform(df[column])

# Define features (X) and target variable (y)
X = df.drop('income', axis=1)  # Features
y = df['income']  # Target variable

# Normalize numerical features
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Split the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=42)

# Print the shapes of the training and testing sets
print("Training set shape:", X_train.shape, y_train.shape)
print("Testing set shape:", X_test.shape, y_test.shape)

Training set shape: (26048, 14) (26048,)
Testing set shape: (6513, 14) (6513,)


In [32]:
from sklearn.metrics import roc_auc_score

## Logistic Regression

In [33]:
from sklearn.linear_model import LogisticRegression

model = LogisticRegression()

model.fit(X_train, y_train)

# Predict on the test set
y_pred = model.predict_proba(X_test)[:, 1]

# Calculate accuracy
auc = roc_auc_score(y_test, y_pred)
print("AUC:", auc)

AUC: 0.85676430940089


## XGBoost

In [34]:
from xgboost import XGBClassifier

model = XGBClassifier()

model.fit(X_train, y_train)

# Predict on the test set
y_pred = model.predict_proba(X_test)[:, 1]

# Calculate accuracy
auc = roc_auc_score(y_test, y_pred)
print("AUC:", auc)

AUC: 0.9272358853470467


## Neural Network

In [37]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from keras.models import Sequential
from keras.layers import Dense

# Define the neural network model
model = Sequential()
model.add(Dense(10, input_dim=X_train.shape[1], activation='relu'))
model.add(Dense(1, activation='sigmoid'))

# Compile the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Train the model
model.fit(X_train, y_train, epochs=15, batch_size=16, verbose=1)


Epoch 1/15
1628/1628 [==============================] - 1s 684us/step - loss: 0.4319 - accuracy: 0.7938
Epoch 2/15
1628/1628 [==============================] - 1s 675us/step - loss: 0.3534 - accuracy: 0.8358
Epoch 3/15
1628/1628 [==============================] - 1s 677us/step - loss: 0.3386 - accuracy: 0.8397
Epoch 4/15
1628/1628 [==============================] - 1s 690us/step - loss: 0.3331 - accuracy: 0.8436
Epoch 5/15
1628/1628 [==============================] - 1s 649us/step - loss: 0.3305 - accuracy: 0.8454
Epoch 6/15
1628/1628 [==============================] - 1s 671us/step - loss: 0.3288 - accuracy: 0.8447
Epoch 7/15
1628/1628 [==============================] - 1s 683us/step - loss: 0.3275 - accuracy: 0.8457
Epoch 8/15
1628/1628 [==============================] - 1s 639us/step - loss: 0.3265 - accuracy: 0.8460
Epoch 9/15
1628/1628 [==============================] - 1s 664us/step - loss: 0.3257 - accuracy: 0.8470
Epoch 10/15
1628/1628 [==============================] - 1s 675u

In [40]:
y_pred = model.predict(X_test)

# Calculate accuracy
auc = roc_auc_score(y_test, y_pred)
print("AUC:", auc)

204/204 [==============================] - 0s 604us/step
AUC: 0.9059273440786451
